In [7]:
import json
import subprocess
from typing import List, Tuple

def get_pairs_to_compare(matrix: List[List[int]]) -> Tuple[List[Tuple[int, int]], float]:
    # Path to the Python script
    python_script_path = "asap_runner.py"

    # Prepare the input for the Python script
    input_data = json.dumps({"matrix": matrix})
    
    # Run the Python script using subprocess
    process = subprocess.Popen(
        ['python', python_script_path],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    # Write the input data to the script
    stdout, stderr = process.communicate(input_data.encode())
    
    if process.returncode != 0:
        raise RuntimeError(f"Error running ASAP script: {stderr.decode()}")

    # Parse the output from the Python script
    result = json.loads(stdout.decode())

    # Extract pairs and max_eig
    pairs = [(pair[0], pair[1]) for pair in result.get("pairs", [])]
    max_eig = result.get("max_eig", 0.0)

    return pairs, max_eig

# Example usage
matrix = [[0] * 100 for _ in range(100)]


pairs, max_eig = get_pairs_to_compare(matrix)
print(f"Pairs: {pairs}")
print(f"Max Eigenvalue: {max_eig}")


Pairs: [(1, 3), (53, 86), (41, 61), (49, 65), (3, 32), (1, 17), (31, 80), (22, 91), (19, 24), (53, 99), (16, 83), (59, 95), (62, 63), (11, 29), (14, 73), (29, 82), (10, 74), (40, 59), (61, 80), (38, 84), (45, 78), (26, 52), (0, 32), (2, 68), (10, 85), (0, 13), (8, 47), (38, 91), (8, 21), (21, 82), (18, 40), (9, 64), (41, 91), (11, 45), (21, 59), (65, 96), (46, 91), (29, 89), (9, 35), (31, 67), (13, 29), (39, 44), (24, 25), (29, 60), (68, 69), (17, 72), (8, 92), (20, 82), (3, 57), (69, 73), (32, 34), (16, 57), (27, 38), (19, 43), (7, 39), (27, 49), (83, 88), (56, 58), (10, 24), (31, 34), (36, 46), (79, 83), (56, 99), (54, 82), (58, 68), (22, 51), (12, 60), (15, 97), (1, 85), (3, 64), (14, 61), (25, 30), (23, 37), (21, 71), (63, 98), (13, 97), (50, 78), (0, 55), (7, 71), (28, 43), (42, 90), (14, 62), (6, 78), (87, 95), (26, 41), (16, 94), (42, 94), (36, 70), (44, 66), (14, 76), (48, 80), (63, 75), (16, 77), (5, 62), (23, 35), (77, 93), (33, 57), (4, 41), (11, 81)]
Max Eigenvalue: 0.99979

In [1]:
# Example of running ASAP on CPU
import asap_cpu
import numpy as np
pwc_mat = np.array([[0,1,2,3,1],
                    [1,0,2,3,1],
                    [1,2,0,3,1],
                    [1,2,3,0,1],
                    [10,2,3,1,0]])
N = np.shape(pwc_mat)[0]
asap = asap_cpu.ASAP(N, selective_eig = True)
kl_divs, pairs_to_compare = asap.run_asap(pwc_mat, mst_mode=True)

# Get current score estimation
scores_mean, scores_std = asap.get_scores()



print("Indeces from pwc_mat to compare:")
print(pairs_to_compare)
print("Scores means \n",scores_mean)
print("Scores standard deviaion \n", scores_std)
print("Expected Information Gain \n", kl_divs)


Indeces from pwc_mat to compare:
[[0 1]
 [0 2]
 [1 4]
 [0 3]]
Scores means 
 [-0.16621793 -0.03432645 -0.19113272 -0.23717982  0.59297576]
Scores standard deviaion 
 [0.2863883  0.31506783 0.29302759 0.28949065 0.29409095]
Expected Information Gain 
 [[ 0.          0.          0.          0.          0.        ]
 [ 5.08467836  0.          0.          0.          0.        ]
 [ 5.07482948  5.06737825  0.          0.          0.        ]
 [ 5.07181058  5.06196475  5.05512198  0.          0.        ]
 [ 5.03847373  5.07441324 -1.          5.06636637  0.        ]]


In [ ]:
# Example of running asap_gpu. Note, that due to computational/space optimizations
# selected batch of comparisons might be different from the pure CPU version

import asap_gpu
import numpy as np
pwc_mat = np.array([[0,1,2,3,1],
                    [1,0,2,3,1],
                    [1,2,0,3,1],
                    [1,2,3,0,1],
                    [1,2,3,1,0]])
pairs, scores_mean, scores_std = asap_gpu.ASAP(pwc_mat, mst_mode=True, cuda=True, get_scores = True)

print("Indeces from pwc_mat to compare:")
print(pairs)
print("Scores means \n",scores_mean)
print("Scores standard deviaion \n", scores_std)



In [2]:
# Example of running ASAP-approx on CPU

import asap_cpu
import numpy as np
pwc_mat = np.array([[0,1,2,3,1],
                    [1,0,2,3,1],
                    [1,2,0,3,1],
                    [1,2,3,0,1],
                    [1,2,3,1,0]])
N = np.shape(pwc_mat)[0]
asap = asap_cpu.ASAP(N, selective_eig = True, approx = True)
pairs_to_compare = asap.run_asap(pwc_mat, mst_mode=True)

# Get current score estimation
scores_mean, scores_std = asap.get_scores()

print("Indeces from pwc_mat to compare:")
print(pairs_to_compare)
print("Scores means \n",scores_mean)
print("Scores standard deviaion \n", scores_std)

Indeces from pwc_mat to compare:
[[0 4]
 [0 1]
 [0 2]
 [0 3]]
Scores means 
 [ 0.21775777 -0.03481698 -0.18566486 -0.19490767  0.22731459]
Scores standard deviaion 
 [0.34988997 0.31438954 0.29271232 0.29199316 0.34936609]
